# Pathway class worked example

This notebook shows an example of how the pathway class works.

## Inputs for this notebooks

In [1]:
import numpy as np
import pandas as pd

In [2]:
hospital_performance = pd.read_csv('data/hospital_performance_thrombectomy.csv')
hospital_performance = hospital_performance.iloc[0]

In [3]:
example_results = pd.read_csv('./data/full_outcome_pathway_trial_example_results_for_pathway_worked_example.csv', index_col=0)

# Keep only the first 100 patients:
example_results = example_results.iloc[0:100]

## Set up the results

there are n patients.
each are assigned values for each step in the pathway.
for example their arrival to scan time.
or whether their onset time is known.
the distributions of assigned values follow target hospital performance values.

## Is onset time known?

generate binomial
each patient is independent of all the others
so don't guarantee final proportion exa,ctly matches

Generate:
+ whether each patient has a known onset time

Method:
+ binomial generator. 

Reference values:
+ proportion of all patients with known onset time



In [20]:
f"{hospital_performance['proportion1_of_all_with_onset_known_ivt']:.1%}"

'59.1%'

### Patient grid
![](./pathway_worked_example/grid_onset_known.png)

### Check: is this on target?
![](./pathway_worked_example/binomial_onset_known_ivt.png)

## Sample onset to arrival time

Generate:
+ separate onset to arrival time for each patient with known onset time

Method:
+ lognorm number generator `np.random.lognorm`
+ fudge times above the time limit. Expected number of patients arriving after the limit is generated using `np.random.binomial`.
+ Times are generated for the MT μ and σ 

Reference values:
+ For generation: target hospital performance μ and σ for onset to arrival times within the MT limit
+ For fudge: proportion of subgroup of patients that have known onset time that arrive within the MT limit
+ For checking only: target hospital performance μ and σ for onset to arrival times within the IVT limit


In [29]:
print(f"μ {hospital_performance['lognorm_mu_onset_arrival_mins_mt']:.3f}")
print(f"σ {hospital_performance['lognorm_sigma_onset_arrival_mins_mt']:.3f}")

μ 4.850
σ 0.675
μ 4.637 - for checking
σ 0.552 - for checking


In [27]:
f"{hospital_performance['proportion2_of_mask1_with_onset_to_arrival_on_time_mt']:.1%}"

'83.2%'

In [30]:
print(f"μ {hospital_performance['lognorm_mu_onset_arrival_mins_ivt']:.3f} - for checking")
print(f"σ {hospital_performance['lognorm_sigma_onset_arrival_mins_ivt']:.3f} - for checking")

μ 4.637 - for checking
σ 0.552 - for checking


### Generated time distributions
![](./pathway_worked_example/time_dist_onset_to_arrival.png)

### Patient grid
![](./pathway_worked_example/grid_onset_times.png)

### Check: is this on target?
![](./pathway_worked_example/binomial_onset_to_arrival_ivt.png)
![](./pathway_worked_example/binomial_onset_to_arrival_mt.png)

## Sample arrival to scan time

Generate:
+ separate arrival to scan time for each patient

Method:
+ lognorm number generator `np.random.lognorm`
+ Times are generated in three batches with separate sanity checks:
  + for patients arriving before the IVT limit, use the μ and σ for IVT
  + for patients arriving before the MT limit, use the μ and σ for MT
  + for patients arriving after the MT limit, also use the μ and σ for MT
+ fudge times above the time limit. Expected number of patients arriving after the limit is generated using `np.random.binomial`.

Reference values:
+ target hospital performance μ and σ for arrival to scan times within the MT limit
+ target hospital performance μ and σ for onset to arrival times within the IVT limit
+ for fudge: proportion of subgroup of patients that have known onset time that arrive within the IVT limit
+ for fudge: proportion of subgroup of patients that have known onset time that arrive within the MT limit

In [31]:
print(f"μ {hospital_performance['lognorm_mu_arrival_scan_arrival_mins_ivt']:.3f}")
print(f"σ {hospital_performance['lognorm_sigma_arrival_scan_arrival_mins_ivt']:.3f}")
print(f"μ {hospital_performance['lognorm_mu_arrival_scan_arrival_mins_mt']:.3f}")
print(f"σ {hospital_performance['lognorm_sigma_arrival_scan_arrival_mins_mt']:.3f}")

μ 3.661
σ 0.704
μ 3.804
σ 0.820


In [32]:
print(f"{hospital_performance['proportion3_of_mask2_with_arrival_to_scan_on_time_ivt']:.1%}")
print(f"{hospital_performance['proportion3_of_mask2_with_arrival_to_scan_on_time_mt']:.1%}")

94.9%
98.1%


### Generated time distributions
![](./pathway_worked_example/time_dist_arrival_to_scan.png)

### Patient grid
![](./pathway_worked_example/grid_arrival_to_scan_times.png)

### Check: is this on target?
![](./pathway_worked_example/binomial_arrival_scan_ivt.png)
![](./pathway_worked_example/binomial_arrival_scan_mt.png)

## Combine results to find onset to scan time

Onset to scan time is simply the sum of the onset to arrival and arrival to scan times. Patients with unknown onset times are also given an unknown onset to scan time.

### Combined times
n.b. this shouldn't necessary follow a lognorm distribution but one has been plotted anyway for reference.
![](./pathway_worked_example/time_dist_onset_to_scan.png)

### Patient grid
![](./pathway_worked_example/grid_onset_to_scan_times.png)

### Check: is this on target?
![](./pathway_worked_example/binomial_onset_scan_ivt.png)
![](./pathway_worked_example/binomial_onset_scan_mt.png)

## Is there enough time for treatment?

compare onset to scan time with treatment time limits.

Reference values:
+ Time limits for treatment with IVT: set to 4hr 30min with 15min allowed overrun
+ Time limits for treatment with MT: set to 8hr 00min with 15min allowed overrun
+ Proportions of patients with enough time to treat with IVT and with MT

In [35]:
print(f"{hospital_performance['proportion5_of_mask4_with_enough_time_to_treat_ivt']:.1%}")
print(f"{hospital_performance['proportion5_of_mask4_with_enough_time_to_treat_mt']:.1%}")

100.0%
99.4%


### Patient array
![](./pathway_worked_example/grid_time_for_treatment.png)

### Check: is this on target?
![](./pathway_worked_example/binomial_time_for_ivt.png)
![](./pathway_worked_example/binomial_time_for_mt.png)

## Decide whether thrombolysis is given

of the ones who have enough time to be treated, 

binomial generate whether treated


Generate:
+ whether each patient with enough time left for IVT is chosen for IVT

Method:
+ binomial generator. 

Reference values:
+ proportion of patients with enough time left for IVT who are chosen for IVT

In [36]:
f"{hospital_performance['proportion6_of_mask5_with_treated_ivt']:.1%}"

'42.7%'

### Patient grid
![](./pathway_worked_example/grid_ivt_chosen.png)

### Check: is this on target?
![](./pathway_worked_example/binomial_treated_ivt.png)

## Decide whether thrombectomy is given

similar to thrombolysis but now also account for patients who receive both treatments

binomial to decide how many patients will receive the both or either

then generate match target props

Generate:
+ whether each patient with enough time left for MT is chosen for MT

Method:
+ binomial generator. 
+ WRITE ME and complicated bits to do with IVT+MT vs IVT not MT and MT not IVT

Reference values:
+ proportion of patients with enough time left for MT who are chosen for MT

In [37]:
f"{hospital_performance['proportion6_of_mask5_with_treated_mt']:.1%}"

'4.7%'

### Patient grid
![](./pathway_worked_example/grid_mt_chosen.png)

### Check: is this on target?
![](./pathway_worked_example/binomial_treated_mt.png)

## Sample scan to thrombolysis time

for only patients who are treated, generate, target proportion

Generate:
+ separate scan to needle time for each patient who is treated

Method:
+ lognorm number generator `np.random.lognorm`

Reference values:
+ For generation: target hospital performance μ and σ for scan to needle times

In [33]:
print(f"μ {hospital_performance['lognorm_mu_scan_needle_mins_ivt']:.3f}")
print(f"σ {hospital_performance['lognorm_sigma_scan_needle_mins_ivt']:.3f}")

μ 2.794
σ 1.099


### Generated times
![](./pathway_worked_example/time_dist_scan_to_needle.png)

### Patient grid
![](./pathway_worked_example/grid_scan_to_needle_times.png)

## Sample scan to thrombectomy time

for only patients who are treated, generate, target proportion

Generate:
+ separate scan to needle time for each patient who is treated

Method:
+ lognorm number generator `np.random.lognorm`

Reference values:
+ For generation: target hospital performance μ and σ for scan to puncture times


In [40]:
print(f"μ {hospital_performance['lognorm_mu_scan_puncture_mins_mt']:.3f}")
print(f"σ {hospital_performance['lognorm_sigma_scan_puncture_mins_mt']:.3f}")

μ 4.324
σ 1.265


### Generated times
![](./pathway_worked_example/time_dist_scan_to_puncture.png)

### Patient grid
![](./pathway_worked_example/grid_scan_to_puncture_times.png)

# Summary of results

## Full patient grid

![](./pathway_worked_example/grid_summary.png)



## Sanity check proportions

big masky

many masks stacking

### IVT
![](./pathway_worked_example/grid_masks_ivt.png)
![](./pathway_worked_example/masks_ivt.png)

### MT
![](./pathway_worked_example/grid_masks_mt.png)
![](./pathway_worked_example/masks_mt.png)